In [286]:
import json
import time
import datetime
import requests 
import pandas as pd
import numpy as np
import yaml
from pathlib import Path
import pickle 
import os
import re
from nltk import metrics, stem, tokenize
 


# API Key IlbRy4iSNXBdEQhTyhQLXl68EVS3U2XttiNvLtpW
# 1000 Calls / Hour
# Put API Key in file. Read in API key from file instead of hardcoding in jupyter notebook. 
# http://petstore.swagger.io/
# https://api.open.fec.gov/swagger/

In [43]:
CONFIG_FILE = Path('.')/Path('0-setup')/Path('fec_config.yaml')
with open(CONFIG_FILE, 'r') as f:
    config = yaml.load(f)

In [13]:
api_key = config['fec']['api_key']

# Read In Collected DataFrames

In [302]:
df_candidate_id = pd.read_pickle('candidate_ids.pkl')
df_candidate_id.head()
df_candidate_id.name = np.array([x.lower().replace(",", "") if isinstance(x, str) else x for x in df_candidate_id.name.values])
df_candidate_id.head()


,name,candidate_id,state,incumbent_challenge_full,party_full,office_full,cycles,election_years,last_file_date,load_date
0,aalders timothy noel,S2UT00229,UT,Open seat,CONSTITUTION PARTY,Senate,"[2012, 2014, 2016, 2018]","[2012, 2018]",2018-04-23,2018-08-01T00:13:22
1,aanestad samuel,H2CA01110,CA,Challenger,REPUBLICAN PARTY,House,"[2012, 2014, 2016]",[2012],2012-02-22,2013-04-26T09:04:30
2,aarestad david,H8CO06237,CO,Challenger,DEMOCRATIC PARTY,House,[2018],[2018],2017-04-26,2017-08-01T20:57:28
3,aaron laura davis,P80002926,US,Open seat,DEMOCRATIC PARTY,President,"[2006, 2008, 2010, 2012, 2014, 2016]",[2008],2007-03-13,2016-11-17T06:10:48
4,abair peter jon,H0MA01024,MA,Challenger,REPUBLICAN PARTY,House,"[2000, 2002, 2004]",[2000],2000-02-02,2002-04-12T00:00:00


## Select Candidates that have winner results 

#### House Winners, Senate Winners, Current Governors 

In [242]:
os.getcwd()

'C:\\Users\\nicho\\Downloads\\Berkeley\\Fall 2018\\data-x\\tenguins\\notebooks'

In [ ]:
house_winners = pd.read_csv(Path('.')/'..'/'data'/'cleaned'/'Clean_House_Winners.csv')

In [243]:
house_winners = pd.read_csv(Path('.')/'..'/'data'/'cleaned'/'Clean_House_Winners.csv')
senate_winners = pd.read_csv(Path('.')/'..'/'data'/'cleaned'/'Clean_Senate_Winners.csv')
gov_winners = pd.read_csv(Path('.')/'..'/'data'/'cleaned'/'Current_Governors_Clean.csv')

In [244]:
print(len(house_winners.Name))
house_winners.head()

1726


,Unnamed: 0,Date,Party,Name,Chamber,District,State
0,0,2016-11-08,r,bradley byrne,house,1,AL
1,1,2016-11-08,r,martha roby,house,2,AL
2,2,2016-11-08,r,mike rogers,house,3,AL
3,3,2016-11-08,r,robert aderholt,house,4,AL
4,4,2016-11-08,r,mo brooks,house,5,AL


In [245]:
print(len(senate_winners.Name))
senate_winners.head()

147


,Unnamed: 0,Date,Party,Name,Chamber,State
0,0,2017-02-09,r,luther strange,senate,AL
1,1,2018-01-03,d,doug jones,senate,AL
2,2,2016-11-08,r,richard shelby,senate,AL
3,3,2016-11-08,r,lisa murkowski,senate,AK
4,4,2016-11-08,r,john mccain,senate,AZ


In [246]:
print(len(gov_winners.Name))
gov_winners.head()

50


,Unnamed: 0,State,Chamber,Name,Party,Date,End
0,0,idaho,governor,butch otter,r,2007-01-01,2019
1,1,utah,governor,gary herbert,r,2009-08-11,2021
2,2,new york,governor,andrew cuomo,d,2011-01-01,2018
3,3,new mexico,governor,susana martinez,r,2011-01-01,2019
4,4,michigan,governor,rick snyder,r,2011-01-01,2019


#### Condition on these candidates only 

In [327]:
df_candidate_id.head()

,name,candidate_id,state,incumbent_challenge_full,party_full,office_full,cycles,election_years,last_file_date,load_date
0,aalders timothy noel,S2UT00229,UT,Open seat,CONSTITUTION PARTY,Senate,"[2012, 2014, 2016, 2018]","[2012, 2018]",2018-04-23,2018-08-01T00:13:22
1,aanestad samuel,H2CA01110,CA,Challenger,REPUBLICAN PARTY,House,"[2012, 2014, 2016]",[2012],2012-02-22,2013-04-26T09:04:30
2,aarestad david,H8CO06237,CO,Challenger,DEMOCRATIC PARTY,House,[2018],[2018],2017-04-26,2017-08-01T20:57:28
3,aaron laura davis,P80002926,US,Open seat,DEMOCRATIC PARTY,President,"[2006, 2008, 2010, 2012, 2014, 2016]",[2008],2007-03-13,2016-11-17T06:10:48
4,abair peter jon,H0MA01024,MA,Challenger,REPUBLICAN PARTY,House,"[2000, 2002, 2004]",[2000],2000-02-02,2002-04-12T00:00:00


In [314]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process


In [ ]:
df_selected_candidates = pd.DataFrame(columns=df_candidate_id.columns)
df_candidate_house = df_candidate_id[df_candidate_id.office_full == 'House']
j = 0
for index_h, house_row in house_winners.iterrows():
    for index, row in df_candidate_house.iterrows():
        name = row[0]
        state = row.state
        party = row.party_full
        house_name = house_row.Name
        house_state = house_row.State
        house_party = house_row.Party
        score = fuzz.token_sort_ratio(name, house_name)
        if (score >= 70) & (state == house_state):
            df_selected_candidates = df_selected_candidates.append(row, ignore_index=True)
            print(name, house_name)
            print(i)
    

#fuzz.token_sort_ratio("Catherine Gitau M.", "Gitau Catherine")

byrne bradley roberts bradley byrne
roby martha martha roby
rogers michael mike rogers
aderholt robert b. rep. robert aderholt
brooks mo mo brooks
palmer gary gary palmer
sewell terri a. terri sewell
o'halleran tom tom o'halleran
mcsally martha martha mcsally
grijalva raul m raul grijalva
gosar paul dr. paul gosar
biggs andy mr. andy biggs
schapira david david schweikert
schweikert david david schweikert
gallego ruben ruben gallego
franks trent trent franks
lesko debbie debbie lesko
sinema kyrsten kyrsten sinema
crawford eric alan rick rick crawford
hill james french french hill
womack steve steve womack
westerman bruce bruce westerman
lamalfa doug doug lamalfa
huffman jared jared huffman
garamendi john john garamendi
graham john john garamendi
mcclintock thomas tom mcclintock
thompson mike mr. mike thompson
matsui doris doris matsui
bera amerish ami bera
cook paul paul cook
paul joe paul cook
mcnerney gerald mark jerry mcnerney
denham jeff jeff denham
desaulnier mark mark desaulnier
p

meng grace grace meng
velazquez nydia m. nydia velazquez
jeffries hakeem hakeem jeffries
clarke yvette yvette clarke
nadler jerrold l. jerrold nadler
debono dan dan donovan
donovan dan dan donovan
maloney carolyn b. carolyn maloney
mccarthy carolyn carolyn maloney
espaillat adriano adriano espaillat
crowley joseph joseph crowley
serrano jose e. jose e. serrano
engel eliot l. eliot engel
lowey nita m nita lowey
maloney sean patrick sean patrick maloney
faso john j. mr. john faso
fee john john faso
tonko paul david paul tonko
stefanik elise m. elise stefanik
tenney claudia claudia tenney
engel tom tom reed
reed thomas w tom reed
katko john m john katko
slaughter louise louise slaughter
higgins brian brian higgins
collins christopher c chris collins
owens chris chris collins
butterfield g. k. g. k. butterfield
holding george e mr. george holding
jones walter b walter b. jones jr.
conlon virginia virginia foxx
foxx virginia ann virginia foxx
rouzer david cheston david rouzer
hudson richard

coffman mike mike coffman
coffman mike rep. mike coffman
perlmutter edwin g ed perlmutter
larson john b john larson
courtney joseph d joe courtney
delauro rosa l rosa delauro
himes jim jim himes
esty elizabeth elizabeth esty
carney john john carney
miller jefferson b. jeff miller
miller mike jeff miller
graham gwen gwen graham
crenshaw ander ander crenshaw
brown corrine corrine brown
brown prince corrine brown
desantis ronald d. ron desantis
mica john l. mr. john mica
mills john john mica
posey bill bill posey
grayson alan mark alan grayson
webster daniel daniel webster
webster daniel j daniel webster
nugent richard b rich nugent
bilirakis gus m gus bilirakis
holden david david jolly
jolly david w. david jolly
koller david c david jolly
castor kathy kathy castor
buchanan vernon vern buchanan
feeney tom tom rooney
rooney tom mr. tom rooney
mooney patrick patrick murphy
murphy patrick e patrick murphy
clawson curtis j curt clawson
hastings alcee l alcee hastings
deutsch peter r ted deutc

desjarlais scott hon. scott desjarlais
black diane l mrs. diane black
blackburn marsha mrs. marsha blackburn
cohen stephen ira stephen fincher
fincher steve mr. stephen fincher
cohen stephen ira steve cohen
gohmert louis b. mr. jr. louie gohmert
poe ted ted poe
johnson sam mr. sam johnson
johnson samuel h. sam johnson
ratcliffe john l john ratcliffe
hensarling jeb hon. jeb hensarling
barton joe linus joe barton
culberson john john culberson
brady kevin kevin brady
green alexander al green
lee al al green
armour michael carl michael mccaul
cloud michael michael mccaul
cole michael k michael mccaul
mccaul michael michael mccaul
granger kay kay granger
thornberry fred d mac thornberry
thornberry mac mac thornberry
weber randy randy weber
hinojosa ruben e. ruben hinojosa
ramirez hinojosa ruben ruben hinojosa
o'rourke robert beto beto o'rourke
flores bill bill flores
lee sheila jackson sheila jackson lee
neugebauer randy honorable randy neugebauer
castro joaquin joaquin castro
olson peter g

richmond cedric l. cedric richmond
boustany charles w. dr. jr. charles boustany
fleming john c mr. jr. john fleming
alexander rodney m. mr. rodney alexander
mcallister vance michael vance mcallister
pingree chellie m chellie pingree
ruppersberger c.a. dutch dutch ruppersberger
sarbanes john p. mr. john sarbanes
edwards donna fern donna edwards
hoyer steny steny hoyer
cummings elijah e elijah cummings
van hollen chris chris van hollen
neal richard e mr. richard neal
macgovern john f jim mcgovern
mcgovern james p. mr. jim mcgovern
tsongas nicola s niki tsongas
kennedy joseph p ii joseph p. kennedy iii
kennedy joseph p iii joseph p. kennedy iii
clark katherine katherine clark
capuano michael e mike capuano
lynch stephen stephen lynch
benishek daniel j. dan benishek
amash justin justin amash
camp david lee dave camp
kildee daniel t. dan kildee
levin sander m mr sander levin
miller candice s. candice miller
bentivolio kerry kerry bentivolio
conyers ian john conyers
conyers john iii john con

In [374]:
df_selected_candidates

,name,candidate_id,state,incumbent_challenge_full,party_full,office_full,cycles,election_years,last_file_date,load_date
0,byrne bradley roberts,H4AL01123,AL,Incumbent,REPUBLICAN PARTY,House,"[2014, 2016, 2018]","[2014, 2016, 2018]",2016-11-22,2017-03-02T21:00:15


In [315]:
house_winners.Name.head()


0      bradley byrne
1        martha roby
2        mike rogers
3    robert aderholt
4          mo brooks
Name: Name, dtype: object

In [288]:
df_candidate_id.name.head()

0    aalders timothy noel
1         aanestad samuel
2          aarestad david
3       aaron laura davis
4         abair peter jon
Name: name, dtype: object

In [304]:
df_candidate_id[(df_candidate_id.name.str.contains(''))]
searchfor = house_winners.Name.values
searchfor_reversed = []
for name in searchfor:
    token = name.split()
    if len(token) > 2:
        temp = token.copy()
        token[-1] = temp[0] 
        token[:-1] = temp[1:]
    else:
        token.reverse()
    token2 = ' '.join(token)
    searchfor_reversed.append(token2)
searchfor_reversed = np.array(searchfor_reversed)
'|'.join(searchfor_reversed)
test = df_candidate_id[df_candidate_id.name.str.contains('|'.join(searchfor_reversed))]

In [316]:
pd.Series(searchfor_reversed).head()

0      byrne bradley
1        roby martha
2        rogers mike
3    aderholt robert
4          brooks mo
dtype: object

In [317]:
#'|'.join(searchfor_reversed)
#df_candidate_id[df_candidate_id.name.str.contains('duncan')]

In [313]:
test[test.name.str.contains('hunter')]
#len(test.name)

,name,candidate_id,state,incumbent_challenge_full,party_full,office_full,cycles,election_years,last_file_date,load_date


In [350]:
df_candidate_id[df_candidate_id.name.str.contains('|'.join(['byrne, bradley']))]
df_candidate_id[df_candidate_id.name.str.contains('byrne')]


,name,candidate_id,state,incumbent_challenge_full,party_full,office_full,cycles,election_years,last_file_date,load_date
2341,byrne bradley roberts,H4AL01123,AL,Incumbent,REPUBLICAN PARTY,House,"[2014, 2016, 2018]","[2014, 2016, 2018]",2016-11-22,2017-03-02T21:00:15
2342,byrne brendan thomas jr,S0NJ00068,NJ,Challenger,DEMOCRATIC PARTY,Senate,"[2000, 2002, 2004, 2006, 2008, 2010, 2012, 201...",[2000],2000-05-01,2018-02-17T09:16:20
2343,byrne leslie l,S6VA00101,VA,Challenger,DEMOCRATIC PARTY,Senate,"[1996, 1998]",[1996],1995-12-06,2002-04-03T00:00:00
2344,byrne leslie l,H2VA11081,VA,Challenger,DEMOCRATIC PARTY,House,"[1992, 1994, 1996, 1998]","[1992, 1994]",1994-06-22,2002-04-03T00:00:00
2345,byrne leslie l,H8VA11039,VA,Open seat,DEMOCRATIC PARTY,House,"[2008, 2010, 2012, 2014, 2016, 2018]",[2008],2007-12-14,2008-02-13T22:08:15
2346,byrne mark,H6MO01220,MO,Challenger,REPUBLICAN PARTY,House,"[2006, 2008]",[2006],2006-03-30,2009-04-29T00:00:00
2347,byrne mary radaszewski mrs,H6MO07268,MO,Challenger,REPUBLICAN PARTY,House,[2016],[2016],2016-05-16,2016-07-11T20:58:46
2348,byrnes lawrence william,H8FL14134,FL,Challenger,DEMOCRATIC PARTY,House,"[2008, 2010]",[2008],2007-07-30,2008-04-04T00:00:00
2349,byrnes pam,H4MI07178,MI,Challenger,DEMOCRATIC PARTY,House,"[2014, 2016]",[2014],2014-08-01,2015-04-02T15:52:44


In [5]:
df_committee_id = pd.read_pickle('committee_ids.pkl')
df_committee_id.head()

,index,candidate_ids,committee_id,committee_type,committee_type_full,cycles,designation,designation_full,filing_frequency,first_file_date,last_f1_date,last_file_date,state,candidate_donor_weight
0,0,[P00008052],C00684373,P,Presidential,[2018],P,Principal campaign committee,Q,2018-08-03,2018-08-03,2018-09-09,MA,1.000
1,1,[S8MN00438],C00555748,N,PAC - Nonqualified,[2014],J,Joint fundraising committee,T,2014-01-28,2014-01-28,2014-12-18,DC,1.000
2,30,"[S0MA00109, S2NV00183, S4IN00014, S4ME00055, S...",C00495390,S,Senate,[2012],J,Joint fundraising committee,T,2011-04-15,2011-05-10,2011-10-31,VA,0.125
3,34,"[H6OH04082, H8OH05036]",C00528091,H,House,[2012],J,Joint fundraising committee,T,2012-08-29,2012-08-29,2012-11-29,OH,0.500
4,35,[H6NY25125],C00532879,N,PAC - Nonqualified,"[2012, 2014, 2016, 2018]",J,Joint fundraising committee,Q,2012-10-22,2012-10-22,2018-10-25,DC,1.000


In [7]:
df_comm = pd.read_pickle('committee_id_formatted_v1.pkl')
df_comm.head()

,index,candidate_ids,committee_id,committee_type,committee_type_full,cycles,designation,designation_full,filing_frequency,first_file_date,last_f1_date,last_file_date,state,candidate_donor_weight
0,0,P00008052,C00684373,P,Presidential,[2018],P,Principal campaign committee,Q,2018-08-03,2018-08-03,2018-09-09,MA,1.000
1,1,S8MN00438,C00555748,N,PAC - Nonqualified,[2014],J,Joint fundraising committee,T,2014-01-28,2014-01-28,2014-12-18,DC,1.000
2,30,S0MA00109,C00495390,S,Senate,[2012],J,Joint fundraising committee,T,2011-04-15,2011-05-10,2011-10-31,VA,0.125
2,30,S2NV00183,C00495390,S,Senate,[2012],J,Joint fundraising committee,T,2011-04-15,2011-05-10,2011-10-31,VA,0.125
2,30,S4IN00014,C00495390,S,Senate,[2012],J,Joint fundraising committee,T,2011-04-15,2011-05-10,2011-10-31,VA,0.125


# Functions to Collect Donor Information for Candidates

In [14]:
# Use by_state/by_candidate, by_size/by_candidate
# QUESTION, Does this by_candidate search include donations made to committee's? 

# Use /schedules/schedule_a/by_size/by_candidate/
# Use /schedules/schedule_a/by_state/by_candidate/


# by_size/by_candidate
#  -0    $200 and under
#  -200  $200.01 - $499.99
#  -500  $500 - $999.99
#  -1000 $1000 - $1999.99
#  -2000 $2000 +

def cand_donor_size(candidate_id, cycle): 
    # Returns aggregated totals of contribution amounts that fit within defined groupings

    url_size = 'https://api.open.fec.gov/v1/schedules/schedule_a/\
by_size/by_candidate/\
?per_page=100\
&page=1\
&sort_null_only=false\
&election_full=false\
&cycle={}\
&sort_hide_null=false\
&api_key={}\
&candidate_id={}'.format(cycle, api_key, candidate_id)

    donor_cand_size = requests.get(url_size)
    #print(donor_cand_size.status_code)
    donor_size_result = donor_cand_size.json()['results']
    df_dcand_size = pd.DataFrame(donor_size_result)
    return df_dcand_size

    
    
# Schedule A individual receipts aggregated by contributor state.
# This is an aggregate of only individual contributions. 
def cand_donor_state(candidate_id, cycle): 
    # Returns aggregated totals of contribution amounts that fit within defined groupings
    url_state = 'https://api.open.fec.gov/v1/schedules/schedule_a/\
by_state/by_candidate/\
?per_page=100\
&page=1\
&sort_null_only=false\
&election_full=false\
&cycle={}\
&sort_hide_null=false\
&api_key={}\
&candidate_id={}'.format(cycle, api_key, candidate_id)
    
    donor_cand_state = requests.get(url_state)
    #print(donor_cand_state.status_code)
    donor_state_result = donor_cand_state.json()['results']
    df_dcand_state = pd.DataFrame(donor_state_result)
    return df_dcand_state



In [15]:
# Use /committee/{committee_id}/schedules/schedule_a/by_size/
# Use /committee/{committee_id}/schedules/schedule_a/by_state/
# Extract candidate_ids supported by each committee, use DataFrame B

def comm_donor_size(committee_id):
    url = 'https://api.open.fec.gov/v1/committee/\
{}/schedules/schedule_a/\
by_size/?sort_hide_null=false\
&per_page=100\
&page=1\
&api_key={}\
&sort_null_only=false'.format(committee_id, api_key)
    
    donor_comm_size = requests.get(url)
    #print(donor_comm_size.status_code)
    donor_comm_size = donor_comm_size.json()['results']
    df_dcomm_size = pd.DataFrame(donor_comm_size)
    return df_dcomm_size

def comm_donor_state(committee_id):
    url = 'https://api.open.fec.gov/v1/committee/\
{}/schedules/schedule_a/\
by_state/?sort_hide_null=false\
&per_page=100\
&page=1\
&api_key={}\
&sort_null_only=false'.format(committee_id, api_key)
    
    donor_comm_state = requests.get(url)
    #print(donor_comm_state.status_code)
    donor_comm_state = donor_comm_state.json()['results']
    df_dcomm_state = pd.DataFrame(donor_comm_state)
    return df_dcomm_state



In [16]:
def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

In [17]:
# Input: Candidate Name, Election Type, Date-time of election (Use Pandas Date time Format)
# 1. Will incorporate a funcation call to 
# 2. to retrieve the fraction of committee donations for that candidate

# by_size/by_candidate
#  -0    $200 and under Level 1 
#  -200  $200.01 - $499.99 Level 2 
#  -500  $500 - $999.99 Level 3 
#  -1000 $1000 - $1999.99 Level 4
#  -2000 $2000 + Level 5

a = cand_donor_state('P80001571', 2016)
states = a.state_full


# Returns rows, 1 row per cycle with all features for one candidate. 
def candidate_donor_df(candidate_name, states, df_candidate_id):
    cand_row = df_candidate_id[df_candidate_id.name == candidate_name]
    cand_ids = cand_row.candidate_id.values
    
    j = 0
    
    #Initialize dataframe
    cand_info = ['name', 'cand_id', 'state', 'cycle', 'incumbent', 'office_full', 'party', 'committee_id']
    donation_levels = ['Donation Level 1',
               'Donation Level 2', 
               'Donation Level 3',
               'Donation Level 4',
               'Donation Level 5']
    levels = [0, 200, 500, 1000, 2000]
    columns = np.append(cand_info, states)
    columns = np.append(columns, donation_levels)

    df = pd.DataFrame(columns=columns)
    
    for cand_id in cand_ids:
        #print(cand_id)
    
        cycles = cand_row.cycles.values[0]


        

        #Iterate through candidates cycles, add all cycles as new rows. Append to each column each
        
        #print(cycles)
        for cycle in cycles:
            df_size = cand_donor_size(cand_id, cycle)
            df_state = cand_donor_state(cand_id, cycle)
            if df_size.values.size == 0 or df_state.values.size == 0:
                continue

            donor_states = df_state.state_full

            #Build row of information (donor_states)

            # Idea
            # Create dictionary with states as keys and initialized with 0s as values. 
            initialize_array = np.zeros(len(states))
            state_dict = dict(zip(states, initialize_array))

            for ds in donor_states:            
                # Overwrite select values with information from df_state
                state_dict[ds] = df_state[df_state.state_full == ds].total.values[0]


            # Add size features to state_dict 


            level_array = np.zeros(5)
            level_dict = dict(zip(donation_levels, level_array))

            for level in df_size['size']:
                if level == 0:
                    donation_level = 'Donation Level 1'
                if level == 200:
                    donation_level = 'Donation Level 2'
                if level == 500:
                    donation_level = 'Donation Level 3'
                if level == 1000:
                    donation_level = 'Donation Level 4'
                if level == 2000:
                    donation_level = 'Donation Level 5'
                level_dict[donation_level] = df_size[df_size['size'] == level].total.values[0]


            cand_info_dict = {'name': candidate_name, 
                              'cand_id': cand_id, 
                              'state': cand_row.state.values[0],
                              'cycle': cycle,
                              'incumbent': cand_row.incumbent_challenge_full.values[0],
                              'office_full': cand_row.office_full.values[0],
                              'party': cand_row.party_full.values[0],
                              'committee_id': 0}
            state_dict = merge_two_dicts(cand_info_dict, state_dict)
            state_dict = merge_two_dicts(state_dict, level_dict)
            #return state_dict


            #return state_dict.values()
            df.loc[j] = list(state_dict.values()) # An array of the row information
            j += 1
    return df

                         
                         
        

In [18]:
#Add df_candidate_id input to the previous function for candidate donor information collection function 

def get_committee_candidates(candidate_name, states, df_candidate_id, df_comm):
    cand_row = df_candidate_id[df_candidate_id.name == candidate_name]
    cand_ids = cand_row.candidate_id.values
    j = 0
    
    #Initialize dataframe
    cand_info = ['name', 'cand_id', 'state', 'cycle', 'incumbent', 'office_full', 'party', 'committee_id']
    donation_levels = ['Donation Level 1',
               'Donation Level 2', 
               'Donation Level 3',
               'Donation Level 4',
               'Donation Level 5']
    levels = [0, 200, 500, 1000, 2000]
    columns = np.append(cand_info, states)
    columns = np.append(columns, donation_levels)

    df = pd.DataFrame(columns=columns)
    #print(cand_ids)
    for cand_id in cand_ids:
        comm_rows = df_comm[df_comm.candidate_ids == cand_id]
        #return comm_rows
        
        #for row in comm_rows: 
        for index, row in comm_rows.iterrows(): #Iterate through each row of the dataframe comm_row
            #print('iter')
            candidate_donor_weight = row.candidate_donor_weight
            cycles = row.cycles
            comm_id = row.committee_id
            #Iterate through candidates cycles, add all cycles as new rows. Append to each column each
            #print(cycles)
            for cycle in cycles:
                df_size = comm_donor_size(comm_id)
                df_state = comm_donor_state(comm_id)
                if df_size.values.size == 0 or df_state.values.size == 0:
                    continue

                donor_states = df_state.state_full

                #Build row of information (donor_states)

                # Idea
                # Create dictionary with states as keys and initialized with 0s as values. 
                initialize_array = np.zeros(len(states))
                state_dict = dict(zip(states, initialize_array))

                for ds in donor_states:            
                    # Overwrite select values with information from df_state
                    state_row = df_state[df_state.state_full == ds]
                    state_dict[ds] = state_row.total.values[0] * candidate_donor_weight


                # Add size features to state_dict 


                level_array = np.zeros(5)
                level_dict = dict(zip(donation_levels, level_array))

                for level in df_size['size']:
                    if level == 0:
                        donation_level = 'Donation Level 1'
                    if level == 200:
                        donation_level = 'Donation Level 2'
                    if level == 500:
                        donation_level = 'Donation Level 3'
                    if level == 1000:
                        donation_level = 'Donation Level 4'
                    if level == 2000:
                        donation_level = 'Donation Level 5'
                    size_row = df_size[df_size['size'] == level]
                    level_dict[donation_level] = size_row.total.values[0] * candidate_donor_weight


                cand_info_dict = {'name': candidate_name, 
                                  'cand_id': cand_id, 
                                  'state': row.state,
                                  'cycle': cycle,
                                  'incumbent': 0,
                                  'office_full': 0,
                                  'party': 0,
                                  'committee_id': row.committee_id}
                state_dict = merge_two_dicts(cand_info_dict, state_dict)
                state_dict = merge_two_dicts(state_dict, level_dict)

                #return state_dict


                #return state_dict.values()
                df.loc[j] = list(state_dict.values()) # An array of the row information
                j += 1
    return df
    


## Run This

In [25]:
#Time to Run 
(len(df_candidate_id.name) / 10 ) * 44 / 60 /60

22.751666666666665

In [22]:
cand_info = ['name', 'cand_id', 'state','cycle', 'incumbent', 'office_full', 'party', 'committee_id']
donation_levels = ['Donation Level 1',
       'Donation Level 2', 
       'Donation Level 3',
       'Donation Level 4',
       'Donation Level 5']
columns = np.append(cand_info, states)
columns = np.append(columns, donation_levels)
df = pd.DataFrame(columns=columns)
i = 1
amount = len(df_candidate_id.name) 
print(datetime.datetime.now())
for name in df_candidate_id.name:
    df_name = candidate_donor_df(name, states, df_candidate_id)
    df = pd.concat([df, df_name], sort=False)
    
    df_name_comm = get_committee_candidates(name, states, df_candidate_id, df_comm)
    df = pd.concat([df, df_name_comm], sort=False)

    print('{}/{}'.format(i, amount))
    i+=1
    if i == 10:
        print(datetime.datetime.now())
        break
df

2018-11-04 13:37:02.524029
1/18615
2/18615
3/18615
4/18615
5/18615
6/18615
7/18615
8/18615
9/18615
2018-11-04 13:37:46.935213


,name,cand_id,state,cycle,incumbent,office_full,party,committee_id,Armed Forces Americas,Armed Forces Europe,...,Washington,Wisconsin,West Virginia,Wyoming,Foreign Countries,Donation Level 1,Donation Level 2,Donation Level 3,Donation Level 4,Donation Level 5
0,"AALDERS, TIMOTHY NOEL",S2UT00229,UT,2012,Open seat,Senate,CONSTITUTION PARTY,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4415.00,250.00,2000.0,0.00,0.0
1,"AALDERS, TIMOTHY NOEL",S2UT00229,UT,2018,Open seat,Senate,CONSTITUTION PARTY,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4372.18,1000.00,0.0,1723.23,2400.0
0,"AALDERS, TIMOTHY NOEL",S2UT00229,UT,2012,0,0,0,C00512632,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4415.00,250.00,2000.0,0.00,0.0
1,"AALDERS, TIMOTHY NOEL",S2UT00229,UT,2014,0,0,0,C00512632,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4415.00,250.00,2000.0,0.00,0.0
2,"AALDERS, TIMOTHY NOEL",S2UT00229,UT,2016,0,0,0,C00512632,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4415.00,250.00,2000.0,0.00,0.0
3,"AALDERS, TIMOTHY NOEL",S2UT00229,UT,2018,0,0,0,C00678300,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4372.18,1000.00,0.0,1723.23,2400.0
0,"AANESTAD, SAMUEL",H2CA01110,CA,2012,Challenger,House,REPUBLICAN PARTY,0,0.0,0.0,...,0.0,800.0,0.0,0.0,0.0,6936.76,8675.00,8500.0,18885.50,19500.0
0,"AANESTAD, SAMUEL",H2CA01110,CA,2012,0,0,0,C00513580,0.0,0.0,...,0.0,800.0,0.0,0.0,0.0,6936.76,8675.00,8500.0,18885.50,19500.0
1,"AANESTAD, SAMUEL",H2CA01110,CA,2014,0,0,0,C00513580,0.0,0.0,...,0.0,800.0,0.0,0.0,0.0,6936.76,8675.00,8500.0,18885.50,19500.0
2,"AANESTAD, SAMUEL",H2CA01110,CA,2016,0,0,0,C00513580,0.0,0.0,...,0.0,800.0,0.0,0.0,0.0,6936.76,8675.00,8500.0,18885.50,19500.0


In [ ]:
df.to_pickle('donor_df.pkl')

### GroupBy Reformat

In [21]:
cand_info = ['name', 'cand_id', 'state', 'cycle', 'incumbent', 'office_full', 'party', 'committee_id']
donation_levels = ['Donation Level 1',
               'Donation Level 2', 
               'Donation Level 3',
               'Donation Level 4',
               'Donation Level 5']
sum_columns = np.append(states, donation_levels)
sum_columns

col = ['name','cand_id', 'state', 'cycle', 'incumbent', 'office_full', 'party', 'committee_id']
df_grouped = df.groupby(cand_info)[sum_columns].sum()
df_grouped

Armed Forces Americas  \
name                  cand_id   state cycle incumbent  office_full party              committee_id                          
AALDERS, TIMOTHY NOEL S2UT00229 UT    2012  0          0           0                  C00512632                       0.0   
                                            Open seat  Senate      CONSTITUTION PARTY 0                               0.0   
                                      2014  0          0           0                  C00512632                       0.0   
                                      2016  0          0           0                  C00512632                       0.0   
                                      2018  0          0           0                  C00678300                       0.0   
                                            Open seat  Senate      CONSTITUTION PARTY 0                               0.0   
AANESTAD, SAMUEL      H2CA01110 CA    2012  0          0           0                  C00513580                       0.0   
                                            Challenger House       REPUBLICAN PARTY   0                               0.0   
                                      2014  0          0           0                  C00513580                       0.0   
                                      2016  0          0           0                  C00513580                       0.0   
AARESTAD, DAVID       H8CO06237 CO    2018  0          0           0                  C00637678                       0.0   
                                            Challenger House       DEMOCRATIC PARTY   0                               0.0   
ABAIR, PETER JON      H0MA01024 MA    2000  Challenger House       REPUBLICAN PARTY   0                               0.0   
                                      2002  Challenger House       REPUBLICAN PARTY   0                               0.0   
ABATE, CAMILLE M      H6NJ05155 NJ    2006  Challenger House       DEMOCRATIC PARTY   0                               0.0   
                                      2008  Challenger House       DEMOCRATIC PARTY   0                               0.0   
ABATE, FRANK G        H2NJ12036 NJ    1992  Challenger House       DEMOCRATIC PARTY   0                               0.0   
ABBAS, JEFFREY LYN    H0IA03071 IA    1990  Challenger House       REPUBLICAN PARTY   0                               0.0   

                                                                                                    Armed Forces Europe  \
name                  cand_id   state cycle incumbent  office_full party              committee_id                        
AALDERS, TIMOTHY NOEL S2UT00229 UT    2012  0          0           0                  C00512632                     0.0   
                                            Open seat  Senate      CONSTITUTION PARTY 0                             0.0   
                                      2014  0          0           0                  C00512632                     0.0   
                                      2016  0          0           0                  C00512632                     0.0   
                                      2018  0          0           0                  C00678300                     0.0   
                                            Open seat  Senate      CONSTITUTION PARTY 0                             0.0   
AANESTAD, SAMUEL      H2CA01110 CA    2012  0          0           0                  C00513580                     0.0   
                                            Challenger House       REPUBLICAN PARTY   0                             0.0   
                                      2014  0          0           0                  C00513580                     0.0   
                                      2016  0          0           0                  C00513580                     0.0   
AARESTAD, DAVID       H8CO06237 CO    2018  0          0           0                  C00637678                     0.0   


In [ ]:
df_grouped.to_pickle('final_donor_df.pkl')